## Experimentation Notebook

This notebook is part of the sandbox and is intended to experiment and play around with the REPUBLIC elasticsearch functionalities.

In [1]:
# This reload library is just used for developing REPUBLIC code modules.
# it can be removed if you're only using the REPUBLIC code.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


## Initialise Republic Elasticsearch Instance

This creates a RepublicElasticsearch object that contains an elasticsearch instance for the Republic CAF indexes, as well as a range of retrieval functions.

Check the [README](https://github.com/HuygensING/republic-project#readme) for configuration details that should be placed in `settings.py`.

In [2]:
from republic.elastic.republic_elasticsearch import initialize_es

rep_es = initialize_es(host_type='external', timeout=60)


## Keyword in Context

A simple way to start exploring is with the `keyword_in_context` function. It takes words or phrases as input and shows a number of hits with surrounding context in the resolutions.

The `keyword_in_context` function returns `hit`s, which are dictionaries with the search `term`, the `pre` and `post` contextual words, a formatted `context`, as well as the `para_id` and `resolution_id` and `resolution_offset` and `para_offset`.

In [3]:
# use single word or multi-word phrase
for hit in rep_es.keyword_in_context("voornoemde Procureur"):
    print(hit["context"])


                       waar by den voornoemde Procureur van Kervel versoekt
              en consenteerende de voornoemde Procureur Alsche in de
                 defensie; waar op voornoemde Procureur van Alphen wyders
                        waar by de voornoemde Procureur van Alphen versogt
               condemnatie, en den voornoemde Procureur de Byo consenteerde
                       waar by den voornoemde Procureur van Son versogt
   condemnatie. Consenteerende den voornoemde Procureur van Kervel in
         en defensie ; versoekende voornoemde Procureur van Son wyders
                       by welke de voornoemde Procureur van Son, alvoorens
                        waar op de voornoemde Procureur van Son condemnatie
                       waar by den voornoemde Procureur van Son versogt
                      wyse door de voornoemde Procureur van Son soude
             qualificeeren, om aan voornoemde Procureur van Son, of
               te authoriseeren de voornoemde Procureur 

The `keyword_in_context` function also has several optional arguments to control the size of the context window (`context_size`, default is 3 words before and after), the number of hits (`num_hits`, default is 10) and query filters to constrain the search space (`filters`, which are added to the query).

**Note**: the `num_hits` argument controls the number of _resolutions_ that are retrieved. Within a resolution, the search keyword may appear multiple times. A context is created for each occurrence of the search keyword, so the number returned of contexts can be (and typically is) higher than the number of hits.

In [4]:
# use context_size to get fewer or more surrounding words as context
for hit in rep_es.keyword_in_context("voornoemde Procureur", context_size=5):
    print(hit["context"])

                              andere zyde; waar by den voornoemde Procureur van Kervel versoekt obedientie, en
                  op condemnatie; en consenteerende de voornoemde Procureur Alsche in de versogte condemnatie
                         exceptie en defensie; waar op voornoemde Procureur van Alphen wyders versoekt condemnatie
                               andere zyde; waar by de voornoemde Procureur van Alphen versogt obedientie: en
                    wyders versogt condemnatie, en den voornoemde Procureur de Byo consenteerde in de
                              andere zyde; waar by den voornoemde Procureur van Son versogt obedientie, en
           Son versogt condemnatie. Consenteerende den voornoemde Procureur van Kervel in de versogte
          behoudens exceptie en defensie ; versoekende voornoemde Procureur van Son wyders daar op
                              andere zyde, by welke de voornoemde Procureur van Son, alvoorens eisch te
                               en defens

In [5]:
for hit in rep_es.keyword_in_context("voornoemde Procureur", context_size=5):
    # First, show paragraph id (which contains session date)
    print(hit["resolution_id"])
    # Second, show the keyword in context
    print(hit["context"])
    # Finally, add newline for readability
    print()


session-1779-12-06-num-1-resolution-15
                              andere zyde; waar by den voornoemde Procureur van Kervel versoekt obedientie, en

session-1779-12-06-num-1-resolution-15
                  op condemnatie; en consenteerende de voornoemde Procureur Alsche in de versogte condemnatie

session-1780-07-18-num-1-resolution-8
                         exceptie en defensie; waar op voornoemde Procureur van Alphen wyders versoekt condemnatie

session-1780-10-13-num-1-resolution-7
                               andere zyde; waar by de voornoemde Procureur van Alphen versogt obedientie: en

session-1780-10-13-num-1-resolution-7
                    wyders versogt condemnatie, en den voornoemde Procureur de Byo consenteerde in de

session-1779-07-23-num-1-resolution-11
                              andere zyde; waar by den voornoemde Procureur van Son versogt obedientie, en

session-1779-07-23-num-1-resolution-11
           Son versogt condemnatie. Consenteerende den voornoemde Pro

In [6]:
# use num_hits to get fewer or more results
for hit in rep_es.keyword_in_context("voornoemde Procureur", context_size=5, num_hits=20):
    print(hit["resolution_id"])
    print(hit["context"])
    print()


session-1779-12-06-num-1-resolution-15
                              andere zyde; waar by den voornoemde Procureur van Kervel versoekt obedientie, en

session-1779-12-06-num-1-resolution-15
                  op condemnatie; en consenteerende de voornoemde Procureur Alsche in de versogte condemnatie

session-1780-07-18-num-1-resolution-8
                         exceptie en defensie; waar op voornoemde Procureur van Alphen wyders versoekt condemnatie

session-1780-10-13-num-1-resolution-7
                               andere zyde; waar by de voornoemde Procureur van Alphen versogt obedientie: en

session-1780-10-13-num-1-resolution-7
                    wyders versogt condemnatie, en den voornoemde Procureur de Byo consenteerde in de

session-1779-07-23-num-1-resolution-11
                              andere zyde; waar by den voornoemde Procureur van Son versogt obedientie, en

session-1779-07-23-num-1-resolution-11
           Son versogt condemnatie. Consenteerende den voornoemde Pro

In [7]:
# use filters to contrain the search space
filters = [
    {"match": {"metadata.session_year": 1672}}
]

for hit in rep_es.keyword_in_context("de Witt", filters=filters):
    print(hit["para_id"])
    print(hit["context"])

session-1672-06-08-num-1-para-54
               vanden Heer Cornen. de Witt, haer Ho:Mo
session-1672-06-08-num-1-para-54
                 vanden geme. heer de Witt, en dat hij
session-1672-06-08-num-1-para-54
                  aenden gein Heer de Witt sal werden gerescribeert
session-1672-06-08-num-1-para-54
                      dat hij heer de Witt, mits sijn indispositie
session-1672-03-14-num-1-para-21
              missive vande Heeren de Witt, ende van Vrijbergen
session-1672-03-01-num-1-para-34
              missive vande Heeren de Witt, ende van Vrijbergen
session-1672-02-18-num-1-para-56
              missive vande Heeren de Witt, van Vrijbergen ende
session-1672-02-25-num-1-para-38
              Missive vande Heeren de Witt, van Vrijbergen ende
session-1672-03-18-num-1-para-10
              missive vande Heeren de Witt, ende van Vrijbergen
session-1672-01-15-num-1-para-16
            Heer Raet Pensionnaris de Witt heeft ter Vergaderinge
session-1672-02-25-num-1-para-39
      

In [8]:
# use filters to contrain the search space
filters = [
    {"range": {"metadata.session_year": {"gte": 1705, "lte": 1710}}}
]

for hit in rep_es.keyword_in_context("Vloot", filters=filters):
    print(hit["para_id"], '\n')
    print(hit["context"], '\n')

session-1710-11-21-num-1-para-31 

               in de gecombineerde Vloot van Engelandt ende 

session-1710-11-21-num-1-para-31 

         gecommandeert hebbende de Vloot van den Staet 

session-1710-11-21-num-1-para-31 

                  met de Engelsche Vloot, successivelijck gecommandeert geweest 

session-1710-11-21-num-1-para-31 

                haer Hoogh Mogende Vloot met die van 

session-1710-11-24-num-1-para-57 

               in de gecombineerde Vloot van Engelandt ende 

session-1710-11-24-num-1-para-57 

         gecommandeert hebbende de Vloot van den Staet 

session-1710-11-24-num-1-para-57 

                  met de Engelsche Vloot, succefsivelijck gecommandeert geweest 

session-1710-11-24-num-1-para-57 

                haer Hoogh Mogende Vloot met die van 

session-1706-04-29-num-1-para-37 

              van de Gecombineerde Vloot; versoeckende, dewyle hy 

session-1706-04-29-num-1-para-37 

                      op 's Landts Vloot, den Secretaris Hubert 

sess

In [9]:
# use filters to contrain the search space
filters = [
    {"match": {"metadata.session_year": 1672}}
]

for hit in rep_es.keyword_in_context("Vlooten", filters=filters, context_size=20):
    print(hit["para_id"], hit["resolution_offset"], '\n')
    print(hit["context"], '\n')


session-1672-05-31-num-1-para-2 0 

Ontfangen een missive vanden Heer Cornelis de Witt, hare Ho:Mo: Gedepden. ende Gevolmachtichde op 's Lants Vlooten in de jegenwoordige expeditie ter Zee, Jehan ‛s Lants Schip de seven Provincien, laverende voor Walcheren, Brugge & Oost van haer 

session-1672-09-01-num-1-para-14 583 

advertentie ten spoedichsten kennisse sal werden gegeven aenden Lieutenant Admirael de Ruijter om daerop behoorlicke reflexie te nemen, de Vijantlicke vlooten te doen observeren, ingevolge van hare Ho:Mo: resolutie vanden seven„ thienden Augusti laestleden, de desseijnen vande Vijanden vanden Staet 

session-1672-09-01-num-1-para-14 1366 

Welderen, ende Lieutenant Admirael de Ruijter sal werden, aengeschreven, dat deselve haer soo veel mogelick op de voor„ schreve Vijantlicke Vlooten sullen informeren, haer Ho:Mo: sonder eenich tijt versuijm, adverteren vande condtschappen die haer vande voornoemde Vijantlicke Vlooten souden mogen 

session-1672-07-15-num-1-para-10 29 

## Retrieving Resolutions

The `rep_es` object has a range of functions to retrieve `resolution` objects.

You can find all available properties and methods of `resolution` objects in `republic_document_model.py`: i.e. in the
[Resolution](https://github.com/HuygensING/republic-project/blob/bb4cdad7b4cb9fb71378d0dde000fe7725ceb45e/republic/model/republic_document_model.py#L392) class, which inherits several properties and methods from the [ResolutionElementDoc](https://github.com/HuygensING/republic-project/blob/bb4cdad7b4cb9fb71378d0dde000fe7725ceb45e/republic/model/republic_document_model.py#L158)

In [13]:
resolutions = rep_es.retrieve_resolutions_by_session_date("1672-02-12")
for res in resolutions:
    print(res.session_date.isoformat(), res.id)

1672-02-12 session-1672-02-12-num-1-attendance_list
1672-02-12 session-1672-02-12-num-1-resolution-1
1672-02-12 session-1672-02-12-num-1-resolution-2
1672-02-12 session-1672-02-12-num-1-resolution-3
1672-02-12 session-1672-02-12-num-1-resolution-4
1672-02-12 session-1672-02-12-num-1-resolution-5
1672-02-12 session-1672-02-12-num-1-resolution-6
1672-02-12 session-1672-02-12-num-1-resolution-7
1672-02-12 session-1672-02-12-num-1-resolution-8
1672-02-12 session-1672-02-12-num-1-resolution-9
1672-02-12 session-1672-02-12-num-1-resolution-10
1672-02-12 session-1672-02-12-num-1-resolution-11
1672-02-12 session-1672-02-12-num-1-resolution-12
1672-02-12 session-1672-02-12-num-1-resolution-13
1672-02-12 session-1672-02-12-num-1-resolution-14
1672-02-12 session-1672-02-12-num-1-resolution-15


In [26]:
query = {
    "bool": {
        "must": [
            {"match": {"metadata.session_year": 1672}},
            {"match": {"paragraphs.text": "raet pensionaris"}}
        ]
    }
}

resolutions = rep_es.retrieve_resolutions_by_query(query)

for res in resolutions:
    print(res.id)
    for para in res.paragraphs:
        print(f"\t{para.text}\n")
    print('--------------------\n')

session-1672-10-14-num-1-attendance_list
	Veneris den 14e. October 1672.

	l. Can

	Præside d'Heer van Bowelt,

	Præsentibus de Heeren Raet Pensionaris Fagel, Reijgersbergh, Vrijbergen, Mauregnault, Kann, Ghemmenich, Bootsma,

	Horenken, Gockinga, Ceck,

	De resolutien op gisteren genomen, sijn gelesen ende geresumeert, gelijck oock gelesen ende gearresteert sijn de depesches daeruijt resulterende.

--------------------

session-1672-02-23-num-1-attendance_list
	Martis den 23en. Februarij 1672.

	Preside d'Heer van Gent

	Præsentibus de Heeren van Gellicom,

	Braseell, Ripperda tot Buirse, Schimmelpenningh, Ommeren,

	Bosvelt, Hoofft, Merens, Raet Pensionaris de Witt, Odijck, Reijgersbergh, Crommon, Vrijbergen, Staetvenisse, Mauregnault,

	Schade, vander Hoolck, met twee Extraordinaris Gedeputeerden van Utrecht.

	Isbrandt van Vierssen, Bouritius, Arnout van Vierssen. Coeverden, Ter Borch, Tengnegel, Gockinga.

	De resolutien gisteren genomen sijn gelesen ende geresumeert, gelijck oock

In [ ]:
# import Counter to do some simple word counting and frequency comparison
from collections import Counter
import re



In [38]:
query = {
    "bool": {
        "must": [
            {"match": {"metadata.session_year": 1672}}
        ]
    }
}

#random_resolutions = rep_es.retrieve_resolutions_by_query(query, size=10000)

all_word_freq = Counter()

for res in random_resolutions:
    for para in res.paragraphs:
        all_word_freq.update([word for word in re.split(r"\W+", para.text) if word != ''])

for word, freq in all_word_freq.most_common(10):
    print(f"{word: <20}{freq: >6}")

ende                 27383
van                  24362
de                   21667
te                   15357
dat                  11430
den                   9961
haer                  8680
tot                   8659
vande                 8619
in                    8232


In [37]:
query = {
    "bool": {
        "must": [
            {"match": {"metadata.session_year": 1672}},
            {"match": {"paragraphs.text": "raet pensionaris"}}
        ]
    }
}

resolutions = rep_es.retrieve_resolutions_by_query(query)


word_freq = Counter()

for res in resolutions:
    for para in res.paragraphs:
        word_freq.update([word for word in re.split(r"\W+", para.text) if word != ''])

rel_freq = {}
min_freq = 3
for word, freq in word_freq.most_common():
    if freq < min_freq:
        continue
    rel_freq[word] = freq / all_word_freq[word]
    
for word in sorted(rel_freq, key = lambda w: rel_freq[w], reverse=True):
    print(f"{word: <20}{rel_freq[word]: >6.4f}{word_freq[word]: >6}{all_word_freq[word]: >8}")

Pensionaris         0.3529     6      17
Wijtingh            0.2000     3      15
Fagel               0.0420     5     119
raet                0.0329     5     152
Gockinga            0.0209     4     191
gecommuniceert      0.0201     3     149
Reijgersbergh       0.0198     4     202
Pensionnaris        0.0197     3     152
Isbrandt            0.0194     3     155
Coeverden           0.0186     3     161
Schepenen           0.0173     3     173
Vierssen            0.0167     4     239
daeruijt            0.0154     4     260
gisteren            0.0148     4     271
resulterende        0.0147     4     272
geresumeert         0.0146     7     480
Mauregnault         0.0144     4     277
Raet                0.0142    14     985
Præside             0.0137     3     219
depesches           0.0136     4     294
geaddresseert       0.0131     4     306
Præsentibus         0.0122     3     245
saken               0.0116     3     259
Vrijbergen          0.0110     4     362
gearresteert    

## Retrieving Aggregate Statistics

You can also directly query the indexes using the elasticsearch instance inside the `rep_es` object, which is stored in the `es_anno` property (so can be addressed via `rep_es.es_anno`).

Below is an example of a query and an aggregation to get the number of resolutions per month in the year 1672:

In [79]:
query = {
    "bool": {
        "must": [
            {"match": {"metadata.session_year": 1672}},
            {"match": {"paragraphs.text": "raet pensionaris"}}
        ]
    }
}

aggs = {
    "months": {
        "date_histogram": {
            "field": "metadata.session_date",
            "calendar_interval": "month"
        }
    }
}


response = rep_es.es_anno.search(index="resolutions", query=query, aggs=aggs, size=0)
buckets = response["aggregations"]["months"]["buckets"]
for bucket in buckets:
    print(bucket["key_as_string"].split("T")[0], bucket["doc_count"])

1672-01-01 78
1672-02-01 26
1672-03-01 85
1672-04-01 14
1672-05-01 93
1672-06-01 47
1672-07-01 65
1672-08-01 59
1672-09-01 59
1672-10-01 74
1672-11-01 60
1672-12-01 33
